In [418]:
class DiatonicChordConfig:
    def __init__(self):
        self.__scale_filter = None
        self.__tonic = None
        self.__dominant = None
        self.__sub_dominant = None
        self.__triad = None
        self.__tetrad = None
        
    @property
    def scale_filter(self):
        # 構成音として使うものを 1, そうでないものを 0 としたリストを格納する。 len=7/12音
        return self.__scale_filter
    @property
    def tonic(self):
        # index番号を格納する
        return self.__tonic
    @property
    def dominant(self):
        # index番号を格納する
        return self.__dominant
    @property
    def sub_dominant(self):
        # index番号を格納する
        return self.__sub_dominant
    @property
    def triad(self):
        # 三和音の場合の付属
        return self.__triad
    @property
    def tetrad(self):
        # 四和音の場合の付属
        return self.__tetrad
    
    def set_parameter(self):
        """
        propertyの値を各クラスの値で更新する
        """
        self.scale_filter()
        self.scale_filter = np.array(self.scale_filter, dtype=bool)
        self.tonic()
        self.dominant()
        self.sub_dominant()
        self.triad()
        self.tetrad()
    
    
class MajorScale(DiatonicChordConfig):
    def __init__(self):
        super().__init__()
        self.set_parameter()

    def scale_filter(self):
        self.scale_filter = [1, 0, 1, 0 , 1, 1, 0, 1, 0, 1, 0, 1]
    def tonic(self):
        # I, Ⅱ, Ⅵ　
        self.tonic = [0, 2, 5]
    def dominant(self):
        self.dominant = [4, 6]
    def sub_dominant(self):
        self.sub_dominant = [3, 1]
    def triad(self):
        self.triad = ["", "m", "m", "", "", "m", "m(♭5)"]
    def tetrad(self):
        self.tetrad = ["M7", "m7", "m7", "M7", "7", "m7", "m7(♭5)"]
         
    def relative_shift(self):
        # 平行調の取得の際にリストをシフトさせるための数
        return 5
     
    
class NaturalMinorScale(DiatonicChordConfig):
    def __init__(self):
        super().__init__()
        self.set_parameter()

    def scale_filter(self):
        self.scale_filter = [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0]
    def tonic(self):
        self.tonic = [0, 2]
    def dominant(self):
        self.dominant = [4, 6]
    def sub_dominant(self):
        self.sub_dominant = [1, 3, 5]
    def triad(self):
        self.triad = ["m", "m(♭5)", "", "m", "m", "", ""]
    def tetrad(self):
        self.tetrad = ["m7", "m7(♭5)", "M7", "m7", "m7", "M7", "7"]
        
    def relative_shift(self):
        # 平行調の取得の際にリストをシフトさせるための数
        return 2
        
        
class HarmonicMinorScale(DiatonicChordConfig):
    """
    ナチュラルマイナーから七度が半音上がる
    """
    def __init__(self):
        super().__init__()
        self.set_parameter()
        
    def scale_filter(self):
        self.scale_filter = [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1]
    def tonic(self):
        self.tonic = [0, 2]
    def dominant(self):
        self.dominant = [4, 6]
    def sub_dominant(self):
        self.sub_dominant = [1, 3, 5]
    def triad(self):
        self.triad = ["m", "m(♭5)", "aug", "m", "", "", "dim"]
    def tetrad(self):
        self.tetrad = ["mM7", "m7(♭5)", "augM7", "m7", "7", "M7", "dim7"]
        
        
class MelodicMinorScale(DiatonicChordConfig):
    """
    メージャースケールと比較して三度が半音上がる
    """
    def __init__(self):
        super().__init__()
        self.set_parameter()
        
    def scale_filter(self):
        self.scale_filter = [1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]
    def tonic(self):
        self.tonic = [0, 2, 5]
    def dominant(self):
        self.dominant = [3, 4, 6]
    def sub_dominant(self):
        self.sub_dominant = [1]
    def triad(self):
        self.triad = ["m", "m", "aug", "", "", "m(♭5)", "m(♭5)"]
    def tetrad(self):
        self.tetrad = ["mM7", "m7", "augM7", "7", "7", "m7(♭5)", "m7(♭5)"]
        
        
class Scales:
    def __init__(self):
        self.all = {
            "Major": MajorScale(),
            "Minor": NaturalMinorScale(),
            "HarmonicMinor": HarmonicMinorScale(),
            "MelodicMinor": MelodicMinorScale(),
        }

        
        self.select = {
            "Major": MajorScale(),
            "Minor": NaturalMinorScale(),
        }
        
        self.major = {
            "Major": MajorScale()
        }
        self.minor = {
            "Minor": NaturalMinorScale(),
            "HarmonicMinor": HarmonicMinorScale(),
            "MelodicMinor": MelodicMinorScale(),
        }

        
m= MajorScale()
mi = NaturalMinorScale()
mm = MelodicMinorScale()
print(m.scale_filter)
m.triad

s = Scales()
s.select["Major"].scale_filter


[ True False  True False  True  True False  True False  True False  True]


array([ True, False,  True, False,  True,  True, False,  True, False,
        True, False,  True])

In [419]:
class Keys:
    def __init__(self):
        self.select = ["C", "C#/D♭", "D", "D#/E♭", "E", "F", "F#/G♭", "G", "G#/A♭", "A", "A#/B♭", "B"]
        self.flat = ["C", "D♭", "D", "E♭", "E", "F", "G♭", "G", "A♭", "A", "B♭", "B"]
        self.sharp = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
        self.all = [self.flat, self.sharp]


In [423]:
class DiatoincKeys:
    def __init__(self, target_key="C", scale_type="Major"):
        self.keys = Keys()
        self.scales = Scales()
        
        self.generate(target_key, scale_type)

    def _rotate_keys(self):
        target_index = self.keys.select.index(self.target_key)
        self.rotated_keys = [self._circulate_keys(keys, target_index) for keys in self.keys.all]
    
    def _circulate_keys(self, keys, target_key_index):
        """
        ターゲットKeyを先頭にしてリスト循環させる
        """
        return np.roll(keys, len(keys)-target_key_index)
    
    
    def _jadge_unique_key(self, pickup_keys):
        """
        白鍵キーが7つ存在している方を返す。なければ♭表記で返す
        """
        for keys in pickup_keys:
            if len({_[0] for _ in keys}) == 7:
                return keys
        else:
            return pickup_keys[0] 
    
    def _get_scale_keys(self):
        """
        指定したスケールのダイアトニック構成音を抜き出す
        """
        self.select_scale = self.scales.select[self.scale_type]
        filter_bool = self.select_scale.scale_filter
        pickup_keys = [keys[filter_bool] for keys in self.rotated_keys]
        self.scale_keys = self._jadge_unique_key(pickup_keys)
        
    def _get_relative_keys(self):
        """
        取得したキーとスケールの平行調のダイアトニック構成音を取り出す
        """
        shift = self.select_scale.relative_shift()
        self.relative_keys = self._circulate_keys(self.scale_keys, shift)
    
    
    def generate(self, target_key, scale_type):
        """
        ダイアトニックと平行調の構成キーのリストを更新する
        """
        self.target_key = target_key
        self.scale_type = scale_type
        
        self._rotate_keys()
        self._get_scale_keys()
        self._get_relative_keys()
        
        return [self.scale_keys, self.relative_keys]

    
k = DiatoincKeys("G#/A♭", "Minor")
print(k.scale_keys)
print(k.relative_keys)
print(k.scale_type)

['G#' 'A#' 'B' 'C#' 'D#' 'E' 'F#']
['B' 'C#' 'D#' 'E' 'F#' 'G#' 'A#']
Minor


In [459]:
class DiatnicChords():
    def __init__(self, target_key="C", scale_type="Major"):
        """
        DiatonicKeysのインスタンスを受け取ります
        """
        self.dk = DiatoincKeys(target_key, scale_type)      
        self.scale_roles = {}
        self.relative_roles = {}

    def extract_role(self):
        if self.dk.scale_type == "Major":
            scale_role_items = self.dk.scales.major.items()
            relative_role_items = self.dk.scales.minor.items()
        elif self.dk.scale_type == "Minor":
            scale_role_items = self.dk.scales.minor.items()
            relative_role_items = self.dk.scales.major.items()
            
        # ダイアトニックの役割を取得
        for k, scale in scale_role_items:
            roles = {
                "tonic": [self.dk.scale_keys[idx] for idx in scale.tonic],
                "dominant": [self.dk.scale_keys[idx] for idx in scale.dominant],
                "sub_dominant":[self.dk.scale_keys[idx] for idx in scale.sub_dominant]
            }
            self.scale_roles[k] = roles
            
        # 平行調の役割を取得
        for k, scale in relative_role_items:
            roles = {
                "tonic": [self.dk.relative_keys[idx] for idx in scale.tonic],
                "dominant": [self.dk.relative_keys[idx] for idx in scale.dominant],
                "sub_dominant":[self.dk.relative_keys[idx] for idx in scale.sub_dominant]
            }
            self.relative_roles[k] = roles               
        
g = DiatnicChords("C", "Major")
g.extract_role()

In [460]:
g.scale_roles
# g.dominant
# g.sub_dominant

{'Major': {'tonic': ['C', 'E', 'A'],
  'dominant': ['G', 'B'],
  'sub_dominant': ['F', 'D']}}

In [461]:
g.relative_roles

{'Minor': {'tonic': ['A', 'C'],
  'dominant': ['E', 'G'],
  'sub_dominant': ['B', 'D', 'F']},
 'HarmonicMinor': {'tonic': ['A', 'C'],
  'dominant': ['E', 'G'],
  'sub_dominant': ['B', 'D', 'F']},
 'MelodicMinor': {'tonic': ['A', 'C', 'F'],
  'dominant': ['D', 'E', 'G'],
  'sub_dominant': ['B']}}

In [477]:
ind = list(g.dk.scale_keys).index("A")
g.dk._circulate_keys(g.dk.scale_keys, ind)[::2]

array(['A', 'C', 'E', 'G'], dtype='<U2')